In [1]:
from phase1_model_instantiation import model_setup, func_test

In [2]:
print("Available Qwen models:")
models = model_setup.list_qwen_models()
for name, info in models.items():
    print(f"  {name}: modified {info['last_modified']}")

Available Qwen models:
  qwen-2.5-3b: modified 2025-09-04 22:55:50+00:00


In [4]:
import os
model_path = "/workspace/praxis-research/base-model/qwen-2.5-3b"
print("Files in model directory:")
for file in os.listdir(model_path):
    print(f"  {file}")

# Check if config.json exists and what's in it
config_path = os.path.join(model_path, "config.json")
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        import json
        config = json.load(f)
        print(f"\nConfig model_type: {config.get('model_type', 'MISSING')}")
else:
    print("\n❌ config.json is missing!")

Files in model directory:
  cache

❌ config.json is missing!


In [5]:
# List all blobs in the qwen_models directory

secrets = model_setup.get_secrets()
from azure.storage.blob import BlobServiceClient

connection_string = f"DefaultEndpointsProtocol=https;AccountName={secrets['storage_account_name']};AccountKey={secrets['storage_account_key']};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(secrets['container'])

print("All blobs in qwen_models/:")
blobs = list(container_client.list_blobs(name_starts_with="qwen_models/"))
for blob in blobs:
    print(f"  {blob.name} ({blob.size} bytes)")

All blobs in qwen_models/:
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/07bfe0640cb5a0037f9322287fbfc682806cf672.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/20024bfe7c83998e9aeaf98a0cd6a2ce6306c2f0.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/38047c6284a25427043f0ab040f623a2a20dd093.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/443909a61d429dff23010e5bddd28ff530edda00.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/4783fe10ac3adce15ac8f358ef5462739852c569.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/51b83fbd4e2d363b60235b9c8d494046a83c1cbf.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/67347b23fb4165b652eb6611f5e1f2a06dfcddba8e909df1b2b0b1857bee06c2.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models-

In [7]:
# Load model/tokenizer from the correct cache path
model_path = "/workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1"

model, tokenizer = model_setup.get_qwen_model(model_path)

✅ Using cached Qwen model at /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1
🔄 Loading model and tokenizer from /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Successfully loaded Qwen model /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1


In [8]:
# Start interactive chat
func_test.chat_loop(model, tokenizer, system_text="You are a helpful AI assistant.", max_new_tokens=256)


Interactive QA/chat with Qwen. Type 'exit' or 'quit' to stop.



You:  what is a transformer in machine learning?


Qwen: A transformer is a type of deep learning model that has become very popular for natural language processing (NLP) tasks, though it can be applied to other domains as well. The transformer architecture was introduced by Vaswani et al. in their 2017 paper "Attention Is All You Need."

Key features and components of the transformer include:

1. **Self-Attention Mechanism**: Unlike traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs), transformers use self-attention mechanisms to weigh the importance of different input elements. This allows the model to focus on relevant parts of the input sequence without relying on sequential processing.

2. **Encoder-Decoder Architecture**: Transformers typically consist of an encoder and a decoder. The encoder processes the input sequence, while the decoder generates the output sequence. This makes them suitable for tasks like machine translation where both inputs and outputs are sequences.

3. **Positional Encodin


You:  exit


Goodbye!
